<br>

# Maps

In [ ]:
import os
import shutil
import folium
import fnmatch
import pandas as pd
import geopandas as gpd

In [ ]:
from osgeo import ogr
from folium import plugins
from zipfile import ZipFile

In [ ]:
from traquitanas.geo import layers as tt

In [ ]:
from paths import *
print(project_path)

<br>

## Layer: Nascentes

In [ ]:
def add_lyr_nascentes(shp_path):
    # Input
    gdf = gpd.read_file(os.path.join(shp_path, 'Nascente.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Nascentes',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Layer: Área Drenagem

In [ ]:
def add_lyr_areadrenagem(shp_path):
    # Input
    gdf = gpd.read_file(os.path.join(shp_path, 'area_drenagem.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Área de Contribuição',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Maps

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18
    
    # Create Map
    m = folium.Map(
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,s
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Layers
    m.add_child(tt.add_lyr_google_satellite(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_terrain(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_hybrid(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_streets(min_zoom, max_zoom))

    # Outros Layers
    m.add_child(add_lyr_nascentes(shps_path))
    m.add_child(add_lyr_areadrenagem(shps_path))
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
# Create Map
m = get_map(os.path.join(shps_path, 'Nascente.shp'))

# Results
m.save(os.path.join(maps_path, 'acp-399-01_map.html'))
m